In [122]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
from six.moves import cPickle as pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Conv2D
from keras import optimizers
from keras import metrics
from keras import regularizers

PATH = 'notMNIST.pickle'
NUM_CLASSES = 10
NUM_PIXEL = 28*28

with open(PATH, 'rb') as file:
    save = pickle.load(file)
    x_train = save['train_dataset']
    x_valid = save['valid_dataset']
    x_test = save['test_dataset']
    train_labels = save['train_labels']
    valid_labels = save['valid_labels']
    test_labels = save['test_labels']
    y_train = keras.utils.to_categorical(train_labels, NUM_CLASSES)
    y_valid = keras.utils.to_categorical(valid_labels, NUM_CLASSES)
    y_test = keras.utils.to_categorical(test_labels, NUM_CLASSES)
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)
    

def train_and_evaluate(m, batch_size=100,
                       epochs=1,
                       x_train=x_train,
                       y_train=y_train,
                       x_valid=x_valid,
                       y_valid=y_valid,
                       y_test =y_test,
                       verbose_fit=1
                      ):
    print("=========", m.name, "========")
    m.summary()
    history = m.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=verbose_fit,
                        validation_data=(x_valid, y_valid))
    
    score_train = m.evaluate(x_train, y_train, verbose=0)
    score_valid = m.evaluate(x_valid, y_valid, verbose=0)
    score_test = m.evaluate(x_test, y_test, verbose=0)
    
    print('Train loss:', score_train[0])
    print('Validation loss:', score_valid[0])
    print('Test loss:', score_test[0])
    print('Train accuracy:', score_train[1])
    print('Validation accuracy:', score_valid[1])
    print('Test accuracy:', score_test[1])

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [135]:
def create_logistic():
    m = Sequential(name="logistic regression")
    m.add(Reshape((NUM_PIXEL,), input_shape=(28,28), name="reshape"))
    m.add(Dense(NUM_CLASSES, activation='softmax', name="regression"))
    
    m.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.1),
                  metrics=[metrics.categorical_accuracy])
    return m
train_and_evaluate(create_logistic())

========= logistic regression ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
regression (Dense)           (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Train on 200000 samples, validate on 10000 samples
Epoch 1/1
200000/200000 [==============================] - 6s 30us/step - loss: 0.6805 - categorical_accuracy: 0.8198 - val_loss: 0.6518 - val_categorical_accuracy: 0.8263
Train loss: 0.640728264794
Validation loss: 0.651821215343
Test loss: 0.412278352714
Train accuracy: 0.83029
Validation accuracy: 0.8263
Test accuracy: 0.8924


In [130]:
def create_mlp1():
    m = Sequential(name="mlp 1 hidden")
    m.add(Reshape((NUM_PIXEL,), input_shape=(28,28), name="reshape"))
    m.add(Dense(100, activation='relu'))
    m.add(Dense(NUM_CLASSES, activation='softmax'))
    m.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.1),
                  metrics=[metrics.categorical_accuracy])
    return m

train_and_evaluate(create_mlp1())

========= mlp 1 hidden ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense_115 (Dense)            (None, 100)               78500     
_________________________________________________________________
dense_116 (Dense)            (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Train on 200000 samples, validate on 10000 samples
Epoch 1/1
200000/200000 [==============================] - 7s 36us/step - loss: 0.5512 - categorical_accuracy: 0.8409 - val_loss: 0.4774 - val_categorical_accuracy: 0.8557
Train loss: 0.467214186074
Validation loss: 0.477356828833
Test loss: 0.271998154724
Train accuracy: 0.86276
Validation accuracy: 0.8557
Test accur

In [131]:
def create_logistic_regularized():
    m = Sequential(name="logistic regression l2 regularized")
    m.add(Reshape((NUM_PIXEL,), input_shape=(28,28), name="reshape"))
    m.add(Dense(NUM_CLASSES, activation='softmax', 
                name="regression", 
                kernel_regularizer=regularizers.l2(0.001)))
    
    m.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.1),
                  metrics=[metrics.categorical_accuracy])
    return m

train_and_evaluate(create_logistic_regularized())

========= logistic regression l2 regularized ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
regression (Dense)           (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________
Train on 200000 samples, validate on 10000 samples
Epoch 1/1
200000/200000 [==============================] - 6s 29us/step - loss: 0.7073 - categorical_accuracy: 0.8201 - val_loss: 0.6818 - val_categorical_accuracy: 0.8280
Train loss: 0.670712413702
Validation loss: 0.681768521404
Test loss: 0.442187683439
Train accuracy: 0.830205
Validation accuracy: 0.828
Test accuracy: 0.8932


In [132]:
# overfit and dropout
def create_mlp1_dropout(dropout_rate=0.2):
    m = Sequential(name="mlp1 dropout")
    m.add(Reshape((NUM_PIXEL,), input_shape=(28,28), name="reshape"))
    m.add(Dropout(dropout_rate))
    m.add(Dense(100, activation='relu', ))
    m.add(Dropout(dropout_rate))
    m.add(Dense(NUM_CLASSES, activation='softmax'))
    m.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.1),
                  metrics=[metrics.categorical_accuracy])
    return m
    
x_train_small = x_train[1:1000, :, :]
y_train_small = y_train[1:1000, :]
model = create_mlp1()
train_and_evaluate(model, 
                   x_train=x_train_small,
                   y_train=y_train_small,
                   verbose_fit=False,
                  epochs=30)

model = create_mlp1_dropout(0.5)
train_and_evaluate(model, 
                   x_train=x_train_small,
                   y_train=y_train_small,
                   verbose_fit=False,
                  epochs=30)

========= mlp 1 hidden ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 100)               78500     
_________________________________________________________________
dense_118 (Dense)            (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
Train loss: 0.143104543975
Validation loss: 0.731374007225
Test loss: 0.467138659501
Train accuracy: 0.981981982042
Validation accuracy: 0.7967
Test accuracy: 0.8631
========= mlp1 dropout ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)

In [133]:
from keras.layers import MaxPooling2D

def create_conv_net():
    m = Sequential(name="cnn")
    m.add(Reshape((28,28,1), input_shape=(28,28)))
    m.add(Conv2D(8, kernel_size=(5,5)))
    m.add(MaxPooling2D(pool_size=(4,4)))
    m.add(Reshape((6*6*8,)))
    m.add(Dense(NUM_CLASSES, activation='softmax'))
    m.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=0.1),
                  metrics=[metrics.categorical_accuracy])
    return m

m = create_conv_net()
train_and_evaluate(m)

========= cnn ========
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_5 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 8)           0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 288)               0         
_________________________________________________________________
dense_121 (Dense)            (None, 10)                2890      
Total params: 3,098
Trainable params: 3,098
Non-trainable params: 0
_________________________________________________________________
Train on 200000 samples, validate on 10000 samples
Epoch 1/1
200000/200000 [==============================] - 30s 150

In [126]:
from keras import backend as K
K.image_data_format()

'channels_last'